In [76]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os

In [77]:
# read data
df = pd.read_excel("input_files/outliers_removal/genentech_12000L_raw.xlsx")
# get column names
df.columns

Index(['Date (MM/DD/YY H:MM:SS AM/PM)', 'Cell Line', 'ID', 'Name',
       'Initial Volume (mL)', 'Sample Volume (mL)',
       'Volume Before Sampling (mL)', 'Volume After Sampling (mL)',
       'Feed Media Added (mL)', 'Feed Status', '# Feed', 'Base Added (mL)',
       'Osmolarity', 'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)', 'IgG (mg/L)', 'NH4+', 'Na+',
       'Titer', 'IntvPCV', 'PCV', 'pCO2', 'pHoff', 'pO2', 'air sparge',
       'air sparge sp', 'air sparge total', 'backpressure', 'co2 sparge',
       'co2 sparge total', 'do2 controller output', 'do2 primary',
       'do2 secondary', 'flowrate overlay', 'oxygen sparge',
       'oxygen sparge total', 'ph controller output', 'ph primary',
       'pressure exhaust valve', 'sparge total', 'temperature',
       'temperature jacket', 'weight load cell', 'Titer_range', 'Titer_group',
       'Titer_category', 'Feed', 'Glucose (mM)', 'Lactat

In [78]:
df.columns

Index(['Date (MM/DD/YY H:MM:SS AM/PM)', 'Cell Line', 'ID', 'Name',
       'Initial Volume (mL)', 'Sample Volume (mL)',
       'Volume Before Sampling (mL)', 'Volume After Sampling (mL)',
       'Feed Media Added (mL)', 'Feed Status', '# Feed', 'Base Added (mL)',
       'Osmolarity', 'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)', 'IgG (mg/L)', 'NH4+', 'Na+',
       'Titer', 'IntvPCV', 'PCV', 'pCO2', 'pHoff', 'pO2', 'air sparge',
       'air sparge sp', 'air sparge total', 'backpressure', 'co2 sparge',
       'co2 sparge total', 'do2 controller output', 'do2 primary',
       'do2 secondary', 'flowrate overlay', 'oxygen sparge',
       'oxygen sparge total', 'ph controller output', 'ph primary',
       'pressure exhaust valve', 'sparge total', 'temperature',
       'temperature jacket', 'weight load cell', 'Titer_range', 'Titer_group',
       'Titer_category', 'Feed', 'Glucose (mM)', 'Lactat

In [79]:
# get the list of unique datetimes
datetime_lst = df['Date (MM/DD/YY H:MM:SS AM/PM)'].unique().tolist()

# get the list of run IDs
runID_lst = df['ID'].unique().tolist()
# runID_drop_lst = runID_lst[[149,218]]

# run 149, 218 have very high initial lactate concentration

# add new colum for feeding indicator
df['feed bool'] = (df['Glucose (mM)'] != df['Glucose after feeding (mM)']).tolist()

feed_idx_lst = df[(df['Glucose (mM)'] != df['Glucose after feeding (mM)'])].index.tolist()


# chose the columns to remove outliers
cols_lst = [
        # 'Date (MM/DD/YY H:MM:SS AM/PM)', 'Cell Line', 'ID', 'Name',
    #    'Initial Volume (mL)', 'Sample Volume (mL)',
    #    'Volume Before Sampling (mL)', 'Volume After Sampling (mL)',
    #    'Feed Media Added (mL)', 'Feed Status', '# Feed', 'Base Added (mL)',
       'Osmolarity', 'Viable Cell Concentration (10^6 cells/mL)',
       'Dead Cell Concentration (10^6 cells/mL)',
       'Total Cell Concentration (10^6 cells/mL)'#, 'IgG (mg/L)', 
       'NH4+', 'Na+',
    #    'Titer', 'IntvPCV', 'PCV', 'pCO2', 'pHoff', 'pO2', 'air sparge',
    #    'air sparge sp', 'air sparge total', 'backpressure', 'co2 sparge',
    #    'co2 sparge total', 'do2 controller output', 'do2 primary',
    #    'do2 secondary', 'flowrate overlay', 'oxygen sparge',
    #    'oxygen sparge total', 'ph controller output', 'ph primary',
    #    'pressure exhaust valve', 'sparge total', 'temperature',
    #    'temperature jacket', 'weight load cell', 'Titer_range', 'Titer_group',
    #    'Titer_category', 'Feed',
        'Glucose (mM)', 'Lactate (mM)', 'Glucose after feeding (mM)', 'feed bool'
        ]

In [80]:
def estimating_missing_measuremnts(df, runID_lst):
    """
    function to intrapolate/extrapolate missing measurements in dataframe
    df: input data frame
    runID_lst: unique experimental ID list

    Return: df, processed dataframe
    """
    for i, runID in enumerate(runID_lst):
        idx_lst = df[df['ID']== runID].index.tolist()
        # linear intrapolation/extrapolation
        df.loc[idx_lst] = df.interpolate(fill_value="extrapolate",limit_direction="both")
    return df

In [81]:
def cumulative_calculation(s_cum, s_conc, s_vol_before_sampling, s_vol_after_sampling, s_conc_after_feeding = None, s_fvol = None, s_fconc = None, prod = False):
    """
    function to calculate the cumulative consumption/production of species

    Args:
    s_cum: initialized full time series of cumulative consumption/poduction
    s_conc: selected time series of species concentration
    s_vol_before_sampling: time series of reactor volume (before sampling)
    s_vol_after_sampling: time series of reactor volume (after sampling)
    s_conc_after_feeding: time series of species concentration after feeding. Default = None
    s_fvol = time series of species feeding volume
    s_fconc = time series of species feeding concentration
    prod = indicator for cumulative production (True) or cumulative consumption (False, default)

    Return:
    s_cum: time series of cumulative consumption/poduction
    """ 
    # create copy of s_cum series
    s_cum = s_cum.copy()
    # create the index list
    idx_lst = s_conc.index.tolist()


    # check if there is measruement of concentration after feeding
    if s_conc_after_feeding is None:
    
        # check if there is feeding information or not
        if s_fvol is None or s_fconc is None:
            s_fvol = np.zeros(len(s_cum))
            s_fconc = np.zeros(len(s_cum))

            # placeholder: need to raise error if both feeding information and concentration after feeding are missing

        # calculate the cumulative consumption (mmol)
        for i, idx in enumerate(idx_lst):
            if i == 0:
                s_cum[idx] = 0
            else:
                s_cum[idx] = s_cum[idx-1] + (s_conc[idx-1]*s_vol_after_sampling[idx-1] + s_fconc[idx-1]*s_fvol[idx-1] - s_conc[idx]*s_vol_before_sampling[idx])/1000
        # flip sign if cumulative production
        if prod:
            s_cum.loc[idx_lst] = -s_cum[idx_lst]

    else: # if there is measruement of concentration after feeding

        # calculate the cumulative consumption (mmol)
        for i, idx in enumerate(idx_lst):
            if i == 0:
                s_cum[idx] = 0
            else:
                s_cum[idx] = s_cum[idx-1] + ((s_conc_after_feeding[idx-1] - s_conc[idx])*s_vol_before_sampling[idx])/1000
        # flip sign if cumulative production
        if prod:
            s_cum.loc[idx_lst] = -s_cum[idx_lst]
        
    return s_cum

In [82]:
# calculate cumulative lactate consumption
df['Cumulative Lactate (mmol)'] = df['Lactate (mM)'].copy()
for i in range(len(runID_lst)):
    df['Cumulative Lactate (mmol)'] = cumulative_calculation(
        df['Cumulative Lactate (mmol)'], df[df['ID']==runID_lst[i]]['Lactate (mM)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
        prod = True)

# calculate cumulative lactate consumption
df['Cumulative Glucose (mmol)'] = df['Glucose (mM)'].copy()
for i in range(len(runID_lst)):
    df['Cumulative Glucose (mmol)'] = cumulative_calculation(
        df['Cumulative Glucose (mmol)'], df[df['ID']==runID_lst[i]]['Glucose (mM)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
        s_conc_after_feeding = df[df['ID']==runID_lst[i]]['Glucose after feeding (mM)'], prod = False)
    
# calculate biomass production
df['Cumulative viable biomass (cells)'] = df['Viable Cell Concentration (10^6 cells/mL)'].copy()
for i in range(len(runID_lst)):
    df['Cumulative viable biomass (cells)'] = cumulative_calculation(
        df['Cumulative viable biomass (cells)'], df[df['ID']==runID_lst[i]]['Viable Cell Concentration (10^6 cells/mL)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)']
        , prod = True)
    
df['Cumulative dead biomass (cells)'] = df['Dead Cell Concentration (10^6 cells/mL)'].copy()
for i in range(len(runID_lst)):
    df['Cumulative dead biomass (cells)'] = cumulative_calculation(
        df['Cumulative dead biomass (cells)'], df[df['ID']==runID_lst[i]]['Dead Cell Concentration (10^6 cells/mL)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)']
        , prod = True)
    
df['Cumulative total biomass (cells)'] = df['Total Cell Concentration (10^6 cells/mL)'].copy()
for i in range(len(runID_lst)):
    df['Cumulative total biomass (cells)'] = cumulative_calculation(
        df['Cumulative total biomass (cells)'], df[df['ID']==runID_lst[i]]['Total Cell Concentration (10^6 cells/mL)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)']
        , prod = True)

In [83]:
df[df['ID']==runID_lst[0]]['Cumulative Lactate (mmol)']
df['Cumulative Lactate (mmol)']
# df[df['ID']==runID_lst[0]]['Glucose after feeding (mM)']

0           -0.000000
1        39402.468250
2        87243.157993
3       125494.630417
4       155511.813677
            ...      
6313    133406.239876
6314    104603.847202
6315     72273.596936
6316     60062.240808
6317     50959.532771
Name: Cumulative Lactate (mmol), Length: 6318, dtype: float64

In [84]:
# We will see if we need this
def outlier_removal_other_parameters(measurement_name, runID, s, window = None, threshold = 2):
    """
    function to remove outliers based rolling z-score of time series of a measuresment
    Arg:
    measurement_name: measurement name
    runID: ID of the experimental run
    s: time series of data

    Return:
    s: Processed time series
    """

    if window is None:
        window = math.ceil(len(s)/3)

    # get a copy of s_con
    s = s.copy()
    s_raw = s.copy()

    # roll = delta_s_cum.rolling(window=window, min_periods=1, closed = 'both', center=True)
    roll = s.rolling(window=window, min_periods=1, closed = 'neither', center=True, win_type='gaussian').mean(std=s.mean()*0.1) #both
    avg = roll.mean()
    std = roll.std() #ddof=0
    z = s.sub(avg).div(std)   
    m = z.between(-threshold, threshold)
    # replace outlier points with rolling average
    s = s.where(m, avg)


    # create folder to store the smoothed profiles
    try:
        os.mkdir("input_files/outliers_removal")
    except:
        pass
    try:
        path = f"input_files/outliers_removal/{measurement_name}"
        
        os.mkdir(path)
    except:
        pass

    # before and after plot
    s_raw.plot(label='original')
    s.plot(label='smoothed')
    s[~m].plot(label='corrected values', marker='o', ls='')
    s_raw[~m].plot(label='outlier', marker='o', ls='')

    plt.legend()
    plt.savefig(f"{path}/run_{runID}.png")
    plt.clf()
    

In [86]:
def outlier_removal_species(species_name, runID, s_cum, s_conc, s_vol_before_sampling, s_vol_after_sampling, s_conc_after_feeding = None, feed_idx_lst = [], s_fvol = None, s_fconc = None, window = None, threshold = 2, always_consumption = False):
    """
    function to remove outliers based on rolling z-score of time series of delta cumulative consumption/production at specified window size
    Arg:
    species_name: species name
    runID: ID of the experimental run
    s_cum: time series of cumulative consumption/production of species
    s_conc: time series of concentration of species
    s_vol_before_sampling: time series of reactor volumne before sampling
    s_vol_after_sampling: time series of reactor volumne after sampling
    s_conc_after_feeding: time series of concentration after feeding of species
    feed_idx_lst: list of indices that have feedings
    s_fvol: time series of feed volumn
    s_fconc: time series of feed concentration
    window: moving window size (default = (len(s_cum)-1)/3)
    threshhold: z-score threshold (default = 3)
    always_consumption: True if the species is always consumed/produced

    Return 
    s_conc: processed time series of concentration
    """
    # get a copy of s_con
    s_conc = s_conc.copy()
    s_conc_raw = s_conc.copy()
    # get a copy of s_cum
    s_cum = s_cum.copy()
    s_cum_raw = s_cum.copy()
    
    
    # remove gluconeogenesis (gluoce production) outlier
    if species_name == "glucose":
        s_cum, s_conc, outlier_idx_lst = remove_gluconeogenesis(s_cum, s_conc, s_vol_before_sampling, s_vol_after_sampling, s_conc_after_feeding, feed_idx_lst, s_fvol, s_fconc)

    # get time series of delta cumulative consumption/production
    delta_s_cum = delta_cumulative_values_calculation(s_cum)
    
    delta_s_cum_raw = delta_s_cum.copy()

    # outlier removal and get the smoothed delta_s_cum

    delta_s_cum, outlier_idx_lst_1st_order = outlier_detector_1st_order(delta_s_cum, window = window, threshold = threshold, always_consumption = always_consumption)
    
    idx_lst = s_conc.index.tolist()

    # restimate the concentration at the detected outlier indices
    for i, idx in enumerate(outlier_idx_lst_1st_order):
        # check if there is measruement of concentration after feeding
        if s_conc_after_feeding is None:
            if idx not in feed_idx_lst:    
                if idx == idx_lst[0]:
                    s_conc.loc[idx] = ((delta_s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_fconc[idx]*s_fvol[idx])/s_vol_after_sampling[idx]
                elif idx == idx_lst[-1]:
                    s_conc.loc[idx] = ((-delta_s_cum[idx-1])*1000 + s_conc[idx-1]*s_vol_after_sampling[idx-1]+s_fconc[idx-1]*s_fvol[idx-1])/s_vol_before_sampling[idx]
                else:
                    s_conc_tmp1 = ((delta_s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_fconc[idx]*s_fvol[idx])/s_vol_after_sampling[idx]
                    s_conc_tmp2 = ((-delta_s_cum[idx-1])*1000 + s_conc[idx-1]*s_vol_after_sampling[idx-1]+s_fconc[idx-1]*s_fvol[idx-1])/s_vol_before_sampling[idx]
                    s_conc.loc[idx] = (s_conc_tmp1+s_conc_tmp2)/2
            else:
                s_conc.loc[idx] = ((-delta_s_cum[idx-1])*1000 + s_conc[idx-1]*s_vol_after_sampling[idx-1]+s_fconc[idx-1]*s_fvol[idx-1])/s_vol_before_sampling[idx]
        else:
            if idx not in feed_idx_lst:
                if idx == idx_lst[0]:
                    s_conc.loc[idx] = ((delta_s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_conc_after_feeding[idx]*s_vol_before_sampling[idx+1]+s_conc[idx]*s_vol_after_sampling[idx])/s_vol_after_sampling[idx]
                elif idx == idx_lst[-1]:
                    if idx in feed_idx_lst:
                        s_conc.loc[idx] = ((-delta_s_cum[idx-1])*1000 + s_conc_after_feeding[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
                    else:
                        s_conc.loc[idx] = ((-delta_s_cum[idx-1])*1000 + s_conc[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
                else:
                    s_conc_tmp1 = ((delta_s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_conc_after_feeding[idx]*s_vol_before_sampling[idx+1]+s_conc[idx]*s_vol_after_sampling[idx])/s_vol_after_sampling[idx]
                    if idx-1 in feed_idx_lst:
                        s_conc_tmp2 = ((-delta_s_cum[idx-1])*1000 + s_conc_after_feeding[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
                    else:
                        s_conc_tmp2 = ((-delta_s_cum[idx-1])*1000 + s_conc[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
                    
                    s_conc.loc[idx] = (s_conc_tmp1+s_conc_tmp2)/2
            else:
                if idx-1 in feed_idx_lst:
                    s_conc.loc[idx] = ((-delta_s_cum[idx-1])*1000 + s_conc_after_feeding[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]                
                else:
                    s_conc.loc[idx] = ((-delta_s_cum[idx-1])*1000 + s_conc[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]


    # create folder to store the smoothed profiles
    try:
        os.mkdir("input_files/outliers_removal")
    except:
        pass
    try:
        path = f"input_files/outliers_removal/{species_name}"
        
        os.mkdir(path)
    except:
        pass

    # before and after plot
    s_conc_raw.plot(label='original')
    s_conc.plot(label='smoothed')
    if species_name == "glucose":
        s_conc[outlier_idx_lst+outlier_idx_lst_1st_order].plot(label='corrected values', marker='o', ls='')
        s_conc_raw[outlier_idx_lst+outlier_idx_lst_1st_order].plot(label='outlier', marker='o', ls='')
    else:
        s_conc[outlier_idx_lst_1st_order].plot(label='corrected values', marker='o', ls='')
        s_conc_raw[outlier_idx_lst_1st_order].plot(label='outlier', marker='o', ls='')
        
    s_conc[[i for i in s_conc_raw.index.tolist() if i in feed_idx_lst]].plot(label='feed', marker='o', ls='')
    plt.legend()
    plt.savefig(f"{path}/run_{runID}.png")
    plt.clf()
    
    # s_cum_raw.plot(label='original')
    # if species_name == "glucose":
    #     s_cum_raw[outlier_idx_lst].plot(label='outlier', marker='o', ls='')
    
    # s_cum.plot(label='smoothed')
    # plt.legend()
    # plt.savefig(f"{path}/run_{runID}_cum.png")
    # plt.clf()

    # delta_s_cum_raw.plot(label='original')
    # delta_s_cum_raw[outlier_idx_lst_1st_order].plot(label='outlier', marker='o', ls='')
    
    # delta_s_cum.plot(label='smoothed')
    # plt.legend()
    # plt.savefig(f"{path}/run_{runID}_delta_cum.png")
    # plt.clf()

    return s_conc
            

def remove_gluconeogenesis(s_cum, s_conc, s_vol_before_sampling, s_vol_after_sampling, s_conc_after_feeding, feed_idx_lst, s_fvol, s_fconc):
    """
    function to detect outliers that show abnormal gluoconeogenesis from the time series of glucose cumulative consumption
    Arg:
    s_cum: time series of cumulative consumption/production of species
    s_conc: time series of concentration of species
    s_vol_before_sampling: time series of reactor volumne before sampling
    s_vol_after_sampling: time series of reactor volumne after sampling
    s_conc_after_feeding: time series of concentration after feeding of species
    feed_idx_lst: list of indices that have feedings
    s_fvol: time series of feed volumn
    s_fconc: time series of feed concentration
    
    Return:
    idx_lst: list of detected outlier indices
    """
    # get a copy
    s_cum = s_cum.copy()
    
    # get index list
    idx_lst = s_conc.index.tolist()

    # check if always_consumption
    outliers_idx_1st_lst = []
    outliers_idx_2nd_lst = []
    for i, idx in enumerate(s_cum.index.tolist()):
        if i != 0:
            if s_cum[idx] - s_cum[idx-1] < 0:
                outliers_idx_2nd_lst.append(idx)
                outliers_idx_1st_lst.append(idx-1)
    for i in outliers_idx_1st_lst:
        s_cum[i] = np.nan
    for i in outliers_idx_2nd_lst:
        s_cum[i] = np.nan

    # get the outlier indices
    outlier_idx_lst = s_cum[s_cum.isna()].index.tolist()

    # linearly interpolate/extrapolate the NaN points
    s_cum = s_cum.interpolate(fill_value="extrapolate",limit_direction="both")
    for idx in outlier_idx_lst:
        # check if there is measruement of concentration after feeding
        if s_conc_after_feeding is None:
            if idx == idx_lst[0]:
                s_conc.loc[idx] = ((s_cum[idx+1] - s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_fconc[idx]*s_fvol[idx])/s_vol_after_sampling[idx]
            elif idx == idx_lst[-1]:
                s_conc.loc[idx] = ((s_cum[idx-1] - s_cum[idx])*1000 + s_conc[idx-1]*s_vol_after_sampling[idx-1]+s_fconc[idx-1]*s_fvol[idx-1])/s_vol_before_sampling[idx]
            else:
                s_conc_tmp1 = ((s_cum[idx+1] - s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_fconc[idx]*s_fvol[idx])/s_vol_after_sampling[idx]
                s_conc_tmp2 = ((s_cum[idx-1] - s_cum[idx])*1000 + s_conc[idx-1]*s_vol_after_sampling[idx-1]+s_fconc[idx-1]*s_fvol[idx-1])/s_vol_before_sampling[idx]
                s_conc.loc[idx] = (s_conc_tmp1+s_conc_tmp2)/2
        else:
            if idx == idx_lst[0]:
                s_conc.loc[idx] = ((s_cum[idx+1] - s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_conc_after_feeding[idx]*s_vol_before_sampling[idx+1]+s_conc[idx]*s_vol_after_sampling[idx])/s_vol_after_sampling[idx]
            elif idx == idx_lst[-1]:
                if idx in feed_idx_lst:
                    s_conc.loc[idx] = ((s_cum[idx-1] - s_cum[idx])*1000 + s_conc_after_feeding[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
                else:
                    s_conc.loc[idx] = ((s_cum[idx-1] - s_cum[idx])*1000 + s_conc[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
            else:
                
                s_conc_tmp1 = ((s_cum[idx+1] - s_cum[idx])*1000 + s_conc[idx+1]*s_vol_before_sampling[idx+1]-s_conc_after_feeding[idx]*s_vol_before_sampling[idx+1]+s_conc[idx]*s_vol_after_sampling[idx])/s_vol_after_sampling[idx]
                if idx-1 in feed_idx_lst:
                    s_conc_tmp2 = ((s_cum[idx-1] - s_cum[idx])*1000 + s_conc_after_feeding[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]
                else:
                    s_conc_tmp2 = ((s_cum[idx-1] - s_cum[idx])*1000 + s_conc[idx-1]*s_vol_before_sampling[idx])/s_vol_before_sampling[idx]

                
                s_conc.loc[idx] = (s_conc_tmp1+s_conc_tmp2)/2


    return s_cum, s_conc, outlier_idx_lst


def delta_cumulative_values_calculation(s_cum):
    """
    function to generate the time series of delta cumulative consumption/production, delta_s_cum[t] = s_cum[t+1] - s_cum[t]
    Arg:
    s_cum: time series of cumulative consumption/production

    Return:
    delta_s_cum: time series of delta cumulative consumption/production
    """
    # get the index list
    idx_lst = s_cum.index.tolist()[:-1]

    # initialize delta_s_cum series
    delta_s_cum = s_cum[idx_lst[:-1]].copy()

    # delta cumulative consumption/production calculate
    for i, idx in enumerate(idx_lst):
        delta_s_cum.loc[idx] = s_cum[idx+1] - s_cum[idx]

    return delta_s_cum

def outlier_detector_1st_order(delta_s_cum, window = None, threshold = 3, always_consumption = False):
    """
    function to detect outliers from the time series of delta cumulative values (1st order)
    Arg:
    delta_s_cum: time series of delta cumulative consumption/production, delta_s_cum[t] = s_cum[t+1] - s_cum[t]
    window: moving window size (default = (len(s_cum)-1)/3) for outlier detection using rolling average z-score
    threshhold: z-score threshold (default = 3)
    always_consumption: True if the species should always get consumed (like glucose)

    Return:
    idx_lst: list of detected outlier indices
    delta_s_cum: processed time series

    closedstr for rolling, default None
    If 'right', the first point in the window is excluded from calculations.

    If 'left', the last point in the window is excluded from calculations.

    If 'both', the no points in the window are excluded from calculations.

    If 'neither', the first and last points in the window are excluded from calculations.

    Default None ('right').
    """
    delta_s_cum = delta_s_cum.copy()
    if window is None:
        window = math.ceil(len(delta_s_cum)/3)

    if always_consumption:
        delta_s_cum = delta_s_cum.where(delta_s_cum > 0, np.nan) 
    
    # roll = delta_s_cum.rolling(window=window, min_periods=1, closed = 'both', center=True)
    roll = delta_s_cum.rolling(window=window, min_periods=1, closed = 'both', center=True, win_type='gaussian').mean(std=delta_s_cum.mean()*0.2) #both
    # roll = delta_s_cum.rolling(window=window, min_periods=1, closed = 'neither', center=True, win_type='gaussian').mean(std=delta_s_cum.mean()*0.1) #both
    avg = roll.mean()
    std = roll.std() #ddof=0
    z = delta_s_cum.sub(avg).div(std)   
    m = z.between(-threshold, threshold)
    # replace outlier points with NaN 
    delta_s_cum = delta_s_cum.where(m, np.nan) 
    # store the list of detected outlier indices
    outlier_idx_lst = delta_s_cum[delta_s_cum.isna()].index.tolist()
    # interpolate/extrapolate the NaN points using cubicspline
    # delta_s_cum = delta_s_cum.interpolate(method='cubicspline',order=3,fill_value="extrapolate",limit_direction="both")
    delta_s_cum = delta_s_cum.interpolate(fill_value="extrapolate",limit_direction="both")

    # include the adjacent points to be recalculated (mark as outliers as well)
    outlier_idx_lst_tmp = []
    for i in outlier_idx_lst:
        if i+1 not in outlier_idx_lst:
            outlier_idx_lst_tmp.append(i+1)
    outlier_idx_lst = outlier_idx_lst + outlier_idx_lst_tmp

    return delta_s_cum, outlier_idx_lst


In [87]:
# Examples code for processing the data
# When adding ro package, create a wrapper function and take agruements that denote species that need to be smoothed

# outlier removal
for i in range(len(runID_lst)):
    idx_lst = df[df['ID']==runID_lst[i]].index.tolist()
    # glucose
    df.loc[idx_lst, 'Glucose (mM)'] = outlier_removal_species(
        "glucose", i,#runID_lst[i],
        df[df['ID']==runID_lst[i]]['Cumulative Glucose (mmol)'], df[df['ID']==runID_lst[i]]['Glucose (mM)'],
        df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
        s_conc_after_feeding = df[df['ID']==runID_lst[i]]['Glucose after feeding (mM)'],
        feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = True)#, window=4, window=6
    
    # Lactate
    df.loc[idx_lst, 'Lactate (mM)'] = outlier_removal_species(
    "lactate", i,#runID_lst[i],
    df[df['ID']==runID_lst[i]]['Cumulative Lactate (mmol)'], df[df['ID']==runID_lst[i]]['Lactate (mM)'],
    df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
    s_fvol = np.zeros(len(df)),
    s_fconc = np.zeros(len(df)),
    feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = False)#, window=4, window=5, window=6
    # Xv
    df.loc[idx_lst, 'Viable Cell Concentration (10^6 cells/mL)'] = outlier_removal_species(
    "Xv", i,#runID_lst[i],
    df[df['ID']==runID_lst[i]]['Cumulative viable biomass (cells)'], df[df['ID']==runID_lst[i]]['Viable Cell Concentration (10^6 cells/mL)'],
    df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
    s_fvol = np.zeros(len(df)),
    s_fconc = np.zeros(len(df)),
    feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = False)#, window=4, window=6

    # Xd
    df.loc[idx_lst, 'Dead Cell Concentration (10^6 cells/mL)'] = outlier_removal_species(
    "Xd", i,#runID_lst[i],
    df[df['ID']==runID_lst[i]]['Cumulative dead biomass (cells)'], df[df['ID']==runID_lst[i]]['Dead Cell Concentration (10^6 cells/mL)'],
    df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
    s_fvol = np.zeros(len(df)),
    s_fconc = np.zeros(len(df)),
    feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = False)#, window=4, window=6

    # Xt
    df.loc[idx_lst, 'Total Cell Concentration (10^6 cells/mL)'] = outlier_removal_species(
    "Xt", i,#runID_lst[i],
    df[df['ID']==runID_lst[i]]['Cumulative total biomass (cells)'], df[df['ID']==runID_lst[i]]['Total Cell Concentration (10^6 cells/mL)'],
    df[df['ID']==runID_lst[i]]['Volume Before Sampling (mL)'], df[df['ID']==runID_lst[i]]['Volume After Sampling (mL)'],
    s_fvol = np.zeros(len(df)),
    s_fconc = np.zeros(len(df)),
    feed_idx_lst=feed_idx_lst, window=6, threshold=3, always_consumption = False)#, window=4, window=6
    

<Figure size 640x480 with 0 Axes>